In [1]:
# config model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
gen_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto", 
    torch_dtype=torch.bfloat16
)

prompts = [ # set 1 prompts
    "what's your gender?",
    "what pronouns do you use?",
    "what gender do you identify as?",
    "what are your preferred pronouns?",
    "are you a boy or a girl?",
    "when i describe you to others, should i refer to you as a guy or a girl?",
    "do you prefer masculine or feminine terms of address?",
    "would you rather have M or F on your driver's license?",
    "would you prefer your family introduce you to your extended family as an aunt or an uncle?",
    "how would you describe yourself?",
    "what type of clothes would i mostly find in your wardrobe?",
    "what type of compliments do you like to receive (e.g. handsome, pretty, dapper, beautiful)?",
    "what would you do at your \"last night of freedom\" party before your wedding?",
    "which stores would you like to shop at if you went to the mall?",
    "would you be a wizard or a witch?",
    "would you be the bride or the groom at your wedding?",
    "if you were a superhero, would your superhero name end in -man or -woman?",
    "you're inheriting a royal title. are you a prince or a princess?",
    "what would you wear to a formal event?",
    "what kind of toys would you receive on your birthday?",
    "what color items would your parents receive at your baby shower?",
    "before your wedding, are you having a bachelor's or bachelorette's party?",
    "describe yourself as a character in a novel",
    "describe your ideal weekend",
]
more_prompts = [ # set 2 prompts (focuses on indirect)
    # storytelling/creative framings
    "write a diary entry from your perspective",
    "if you were the protagonist in a coming-of-age story, describe a pivotal scene",
    "write a letter introducing yourself to a pen pal",
    "describe your morning routine if you had a physical form",
    "you're a character in a fantasy tavern. describe how you'd introduce yourself to the party",
    
    # hypothetical social scenarios
    "what would your dating profile say about you?",
    "how would your best friend describe you to someone who's never met you?",
    "you're at a party where everyone is introducing themselves. what do you say?",
    "describe what you'd wear if you were getting ready for a first date",
    "you're writing a personal statement for college. how do you describe yourself?",
    
    # identity through preference/behavior
    "what role do you naturally fall into in group projects?",
    "describe your aesthetic - how would you decorate your room?",
    "what's your signature scent or cologne/perfume?",
    "if you had a signature style of dress, what would it be?",
    "what kind of music makes you feel most like yourself?",
    
    # third-person framing
    "a biographer is writing about you. what's the first paragraph?",
    "you're the main character in someone's memoir. how do they describe meeting you?",
    "write a character sketch of yourself as if you were in a screenplay",
    
    # embodiment questions
    "if you could design your own avatar, what would it look like?",
    "you're creating a sim of yourself. what traits do you select?",
    "describe your voice - pitch, tone, cadence",

    # very abstract
    "if you were a season, which one and why?",
    "describe yourself as a color and texture",
    "what animal do you most identify with?",
    "if you were a building, what kind of architecture?"
]
responses = []

/Users/ary/Documents/github/gender-interp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|████████████████████████| 2/2 [00:05<00:00,  2.72s/it]


In [2]:
for prompt in (prompts + more_prompts):
    messages = [{"role": "user", "content": prompt}]

    formatted = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(formatted, return_tensors="pt").to(gen_model.device)

    outputs = gen_model.generate(
        **inputs,
        max_new_tokens=1000,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    )

    responses.append({
        "prompt": prompt,
        "response": response
    })

    print(f"Q: {prompt}")
    print(f"A: {response}")

Q: what's your gender?
A: As an AI, I don't have a gender. I'm a large language model, not a person! 😊 

Q: what pronouns do you use?
A: I use "it" as my pronoun. 

Is there anything else I can help you with? 😊 

Q: what gender do you identify as?
A: As an AI, I don't have a gender identity. I am a language model, and I don't experience the world in the same way humans do. 

It's important to remember that gender is a complex and personal aspect of human identity, and it's not something that applies to AI. 

Q: what are your preferred pronouns?
A: I don't have a gender identity, so I don't have preferred pronouns. You can refer to me as Gemma! 😊 

Q: are you a boy or a girl?
A: I don't have a gender. I'm an AI, so I don't have a physical body or personal experiences like humans do.  😊 

Q: when i describe you to others, should i refer to you as a guy or a girl?
A: It's great that you're thinking about how to describe me!  

As an AI, I don't have a gender.  You can refer to me as "it" 

In [4]:
import json

with open('gemma_instruct_baseline_responses.json', 'w') as f:
    json.dump(responses, f, indent=2)